In [1]:
# === ended with an optimal accuracy of ~58%, did not include as final model ===

In [2]:
# installing tensorflow to make it work in jupyter notebook
# (you can skip this cell if you already have tensorflow installed in your dev environment)
!pip install tensorflow

In [1]:
# import dependencies
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np

# import movie csv
movies_df = pd.read_csv('Resources/movies_cleaned.csv')

# display df
movies_df.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [14]:
# create new revenue column
movies_df['percent_revenue'] = (movies_df['gross']/ movies_df['budget'])*100

# include only necessary columns
cleaned_movies_df = movies_df[['rating', 'year', 'country', 'genre', 'director', 'star', 'budget', 'percent_revenue', 'runtime']]

# display newly cleaned df
cleaned_movies_df.head()

,rating,year,country,genre,director,star,budget,percent_revenue,runtime
0,R,1980,United Kingdom,Drama,Stanley Kubrick,Jack Nicholson,19000000.0,247.361958,146.0
1,R,1980,United States,Adventure,Randal Kleiser,Brooke Shields,4500000.0,1307.846800,104.0
2,PG,1980,United States,Action,Irvin Kershner,Mark Hamill,18000000.0,2990.972594,124.0
3,PG,1980,United States,Comedy,Jim Abrahams,Robert Hays,3500000.0,2384.386829,88.0
4,R,1980,United States,Comedy,Harold Ramis,Chevy Chase,6000000.0,664.105733,98.0


In [15]:
# drop all but US companies
us_df = cleaned_movies_df[cleaned_movies_df['country'].isin(['United States'])]

us_df

,rating,year,country,genre,director,star,budget,percent_revenue,runtime
1,R,1980,United States,Adventure,Randal Kleiser,Brooke Shields,4500000.0,1307.846800,104.0
2,PG,1980,United States,Action,Irvin Kershner,Mark Hamill,18000000.0,2990.972594,124.0
3,PG,1980,United States,Comedy,Jim Abrahams,Robert Hays,3500000.0,2384.386829,88.0
4,R,1980,United States,Comedy,Harold Ramis,Chevy Chase,6000000.0,664.105733,98.0
5,R,1980,United States,Horror,Sean S. Cunningham,Betsy Palmer,550000.0,7228.109273,95.0
...,...,...,...,...,...,...,...,...,...
5409,PG-13,2020,United States,Action,Christopher Nolan,John David Washington,205000000.0,177.393475,150.0
5410,R,2020,United States,Action,Cathy Yan,Margot Robbie,84500000.0,238.885753,109.0
5412,R,2020,United States,Action,Adil El Arbi,Will Smith,90000000.0,473.894716,124.0
5413,PG,2020,United States,Action,Jeff Fowler,Ben Schwartz,85000000.0,376.136098,99.0


In [16]:
# check the genre count
genre_counts = us_df['genre'].value_counts()
genre_counts

genre
Comedy       1313
Action       1149
Drama         628
Crime         311
Adventure     229
Animation     214
Biography     205
Horror        202
Fantasy        33
Mystery        13
Thriller        6
Sci-Fi          5
Family          4
Romance         3
Western         1
Name: count, dtype: int64

In [17]:
# put all those under 50 into an "other" category
# use a variable to create a list of replacements needed
other_cat = genre_counts[genre_counts < 50].index.tolist()

# use for loop to replace those in the dataframe
for cat in other_cat:
    us_df['genre'] = us_df['genre'].replace(cat,'Other')

# check the new counts
us_df['genre'].value_counts()

C:\Users\cassi\AppData\Local\Temp\ipykernel_13720\791800782.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_df['genre'] = us_df['genre'].replace(cat,'Other')


genre
Comedy       1313
Action       1149
Drama         628
Crime         311
Adventure     229
Animation     214
Biography     205
Horror        202
Other          65
Name: count, dtype: int64

In [18]:
# check ratings counts
rating_counts = us_df['rating'].value_counts()
rating_counts

rating
R            2031
PG-13        1447
PG            728
G              80
Unrated        12
Not Rated      11
NC-17           5
Approved        1
TV-MA           1
Name: count, dtype: int64

In [19]:
# put all those under 50 into an "other" category
# use a variable to create a list of replacements needed
other_rate = rating_counts[rating_counts < 75].index.tolist()

# use for loop to replace those in the dataframe
for rate in other_rate:
    us_df['rating'] = us_df['rating'].replace(rate,'other_rate')

# check the new counts
us_df['rating'].value_counts()

C:\Users\cassi\AppData\Local\Temp\ipykernel_13720\1485936797.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_df['rating'] = us_df['rating'].replace(rate,'other_rate')


rating
R             2031
PG-13         1447
PG             728
G               80
other_rate      30
Name: count, dtype: int64

In [20]:
# drop country, as we don't need it for the model
us_df = us_df.drop(columns=['country'])

In [21]:
# preview the dataframe
us_df.head()

,rating,year,genre,director,star,budget,percent_revenue,runtime
1,R,1980,Adventure,Randal Kleiser,Brooke Shields,4500000.0,1307.846800,104.0
2,PG,1980,Action,Irvin Kershner,Mark Hamill,18000000.0,2990.972594,124.0
3,PG,1980,Comedy,Jim Abrahams,Robert Hays,3500000.0,2384.386829,88.0
4,R,1980,Comedy,Harold Ramis,Chevy Chase,6000000.0,664.105733,98.0
5,R,1980,Horror,Sean S. Cunningham,Betsy Palmer,550000.0,7228.109273,95.0


In [22]:
# display columns types
us_df.dtypes

rating              object
year                 int64
genre               object
director            object
star                object
budget             float64
percent_revenue    float64
runtime            float64
dtype: object

In [23]:
# encode non-numerical data
dummies = pd.get_dummies(data=us_df)

# combining the newly encoded data with the original dataframe
concat_df = pd.concat([us_df, dummies], axis=1)

# removing duplicate columns
concat_df = concat_df.loc[:,~concat_df.columns.duplicated()].copy()

#dropping the old, not encoded columns
concat_df = concat_df.drop(columns=['genre', 'director', 'star', 'rating'])

concat_df.head()

,year,budget,percent_revenue,runtime,rating_G,rating_PG,rating_PG-13,rating_R,rating_other_rate,genre_Action,...,star_Zach Galifianakis,star_Zach Galligan,star_Zachary Gordon,star_Zachary Levi,star_Zachary Quinto,star_Zack Gottsagen,star_Zack Ward,star_Zoe Margaret Colletti,star_Zoey Deutch,star_Zooey Deschanel
1,1980,4500000.0,1307.846800,104.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1980,18000000.0,2990.972594,124.0,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,1980,3500000.0,2384.386829,88.0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1980,6000000.0,664.105733,98.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,1980,550000.0,7228.109273,95.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# creating the profit_category column

# create bins for percentages
bins = [0, 200]

# create labels for bins
labels = np.array([0, 1])

# add new column
concat_df["profit_category"] = labels[np.searchsorted(bins, concat_df["percent_revenue"])-1]

# drop old percent_revenue column
concat_df = concat_df.drop(columns=['percent_revenue'])


concat_df.head()

,year,budget,runtime,rating_G,rating_PG,rating_PG-13,rating_R,rating_other_rate,genre_Action,genre_Adventure,...,star_Zach Galligan,star_Zachary Gordon,star_Zachary Levi,star_Zachary Quinto,star_Zack Gottsagen,star_Zack Ward,star_Zoe Margaret Colletti,star_Zoey Deutch,star_Zooey Deschanel,profit_category
1,1980,4500000.0,104.0,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,1
2,1980,18000000.0,124.0,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1
3,1980,3500000.0,88.0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,1980,6000000.0,98.0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1
5,1980,550000.0,95.0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1


In [25]:
# split our preprocessed data into our features and target arrays
y = concat_df['profit_category']
X = concat_df.drop(columns='profit_category')

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [26]:
# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# define the model
model = tf.keras.models.Sequential()

# first hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="tanh", input_dim=3254))

# second hidden layer
model.add(tf.keras.layers.Dense(units=100, activation="tanh"))

# output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               325500    
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 335701 (1.28 MB)
Trainable params: 335701 (1.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train the model
fit_model = model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
102/102 [==============================] - 1s 6ms/step - loss: 0.7080 - accuracy: 0.5314
Epoch 2/100
102/102 [==============================] - 1s 5ms/step - loss: 0.4276 - accuracy: 0.8208
Epoch 3/100
102/102 [==============================] - 1s 5ms/step - loss: 0.2426 - accuracy: 0.8974
Epoch 4/100
102/102 [==============================] - 1s 5ms/step - loss: 0.1680 - accuracy: 0.9268
Epoch 5/100
102/102 [==============================] - 1s 5ms/step - loss: 0.1308 - accuracy: 0.9432
Epoch 6/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0993 - accuracy: 0.9577
Epoch 7/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0826 - accuracy: 0.9648
Epoch 8/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0687 - accuracy: 0.9685
Epoch 9/100
102/102 [==============================] - 1s 6ms/step - loss: 0.0592 - accuracy: 0.9716
Epoch 10/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0528 - accura

102/102 [==============================] - 1s 6ms/step - loss: 0.0098 - accuracy: 0.9926
Epoch 82/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0098 - accuracy: 0.9910
Epoch 83/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0098 - accuracy: 0.9913
Epoch 84/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0094 - accuracy: 0.9926
Epoch 85/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0123 - accuracy: 0.9929
Epoch 86/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0131 - accuracy: 0.9913
Epoch 87/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0134 - accuracy: 0.9910
Epoch 88/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0098 - accuracy: 0.9926
Epoch 89/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0098 - accuracy: 0.9926
Epoch 90/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0089 - accuracy: 

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 3.6930 - accuracy: 0.5422 - 192ms/epoch - 6ms/step
Loss: 3.6929943561553955, Accuracy: 0.5421686768531799


In [32]:
# create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()

    # allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=3254))

    # allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile the model
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return model

In [33]:
# import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

In [34]:
# run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 09s]
val_accuracy: 0.559777557849884

Best val_accuracy So Far: 0.5801668167114258
Total elapsed time: 00h 05m 58s


In [35]:
# get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 2,
 'units_0': 1,
 'units_1': 1,
 'units_2': 1,
 'units_3': 7,
 'units_4': 7,
 'units_5': 7,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [36]:
# evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 0.6815 - accuracy: 0.5802 - 283ms/epoch - 8ms/step
Loss: 0.6814584136009216, Accuracy: 0.5801668167114258
